In [3]:
# ===============================================================
# FAST CycleGAN for Face <-> Sketch (Optimized for Google Colab)
# ===============================================================

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import itertools

# ===============================================================
# Google Drive
# ===============================================================
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/Dataset_Q1_A-01_Gen-AI'

# ===============================================================
# SPEED-UP OPTION: Use only first N images
# ===============================================================
MAX_IMAGES = 500      # Reduce if still slow


# ===============================================================
# Dataset Class
# ===============================================================
class FaceSketchDataset(Dataset):
    def __init__(self, root_dir, transform=None):

        self.photo_dir = os.path.join(root_dir, 'photos')
        self.sketch_dir = os.path.join(root_dir, 'sketches')

        self.photo_images = sorted([f for f in os.listdir(self.photo_dir) if f.endswith('.jpg')])
        self.sketch_images = sorted([f for f in os.listdir(self.sketch_dir) if f.endswith('.jpg')])

        # Speed: limit N images
        self.photo_images = self.photo_images[:MAX_IMAGES]
        self.sketch_images = self.sketch_images[:MAX_IMAGES]

        self.transform = transform

    def __len__(self):
        return min(len(self.photo_images), len(self.sketch_images))

    def __getitem__(self, idx):
        photo = Image.open(os.path.join(self.photo_dir, self.photo_images[idx])).convert('RGB')
        sketch = Image.open(os.path.join(self.sketch_dir, self.sketch_images[idx])).convert('RGB')

        if self.transform:
            photo = self.transform(photo)
            sketch = self.transform(sketch)

        return {"photo": photo, "sketch": sketch}


# ===============================================================
# Transformations (Speed: 256 → 128)
# ===============================================================
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ===============================================================
# Generators (Speed: residual blocks from 9 → 3)
# ===============================================================

class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, padding=1),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),

            nn.Conv2d(dim, dim, 3, padding=1),
            nn.InstanceNorm2d(dim)
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_blocks=3):  # SPEED OPT
        super().__init__()
        model = [
            nn.Conv2d(input_nc, 64, 7, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(True)
        ]

        # down
        in_f = 64
        out_f = 128
        model += [
            nn.Conv2d(in_f, out_f, 3, stride=2, padding=1),
            nn.InstanceNorm2d(out_f),
            nn.ReLU(True)
        ]

        in_f = out_f
        out_f = 256
        model += [
            nn.Conv2d(in_f, out_f, 3, stride=2, padding=1),
            nn.InstanceNorm2d(out_f),
            nn.ReLU(True)
        ]

        in_f = out_f

        # residual
        for _ in range(n_blocks):
            model += [ResnetBlock(in_f)]

        # up
        out_f = in_f // 2
        model += [
            nn.ConvTranspose2d(in_f, out_f, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(out_f),
            nn.ReLU(True)
        ]

        in_f = out_f
        out_f = in_f // 2
        model += [
            nn.ConvTranspose2d(in_f, out_f, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(out_f),
            nn.ReLU(True)
        ]

        model += [
            nn.Conv2d(out_f, output_nc, 7, padding=3),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


# ===============================================================
# PatchGAN Discriminator
# ===============================================================
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super().__init__()
        model = [
            nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
        ]

        in_f = 64
        out_f = 128
        model += [
            nn.Conv2d(in_f, out_f, 4, stride=2, padding=1),
            nn.InstanceNorm2d(out_f),
            nn.LeakyReLU(0.2),
        ]

        in_f = out_f
        out_f = 256
        model += [
            nn.Conv2d(in_f, out_f, 4, stride=2, padding=1),
            nn.InstanceNorm2d(out_f),
            nn.LeakyReLU(0.2),
        ]

        model += [nn.Conv2d(out_f, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


# ===============================================================
# Device
# ===============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G_A2B = Generator(3, 3).to(device)
G_B2A = Generator(3, 3).to(device)
D_A = Discriminator(3).to(device)
D_B = Discriminator(3).to(device)

# ===============================================================
# Loss & Optimizers
# ===============================================================
adv_loss = nn.MSELoss()
cycle_loss_fn = nn.L1Loss()
id_loss_fn = nn.L1Loss()

optimizer_G = optim.Adam(
    itertools.chain(G_A2B.parameters(), G_B2A.parameters()),
    lr=0.0002, betas=(0.5, 0.999)
)
optimizer_D_A = optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))


# ===============================================================
# Loaders
# ===============================================================
train_dataset = FaceSketchDataset(os.path.join(BASE_PATH, "train"), transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = FaceSketchDataset(os.path.join(BASE_PATH, "val"), transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# ===============================================================
# Checkpoint
# ===============================================================
ckpt_dir = "/content/drive/MyDrive/Checkpoints_CycleGAN_Fast"
os.makedirs(ckpt_dir, exist_ok=True)

best_val = float("inf")

# ===============================================================
# Training
# ===============================================================
EPOCHS = 10          # now you can train more because it's fast
lambda_cycle = 10
lambda_identity = 5

for epoch in range(1, EPOCHS + 1):
    loop = tqdm(train_loader)

    for batch in loop:
        real_A = batch['photo'].to(device)
        real_B = batch['sketch'].to(device)

        # ============================================
        # Train Generators
        # ============================================
        optimizer_G.zero_grad()

        fake_B = G_A2B(real_A)
        fake_A = G_B2A(real_B)

        # GAN Loss
        loss_GAN = adv_loss(D_B(fake_B), torch.ones_like(D_B(fake_B))) + \
                   adv_loss(D_A(fake_A), torch.ones_like(D_A(fake_A)))

        # Cycle loss
        rec_A = G_B2A(fake_B)
        rec_B = G_A2B(fake_A)
        loss_cycle = cycle_loss_fn(rec_A, real_A) + cycle_loss_fn(rec_B, real_B)

        # Identity
        loss_id = id_loss_fn(G_A2B(real_B), real_B) + id_loss_fn(G_B2A(real_A), real_A)

        # Total
        loss_G = loss_GAN + lambda_cycle * loss_cycle + lambda_identity * loss_id
        loss_G.backward()
        optimizer_G.step()

        # ============================================
        # Train D_A
        # ============================================
        optimizer_D_A.zero_grad()
        loss_D_A = (adv_loss(D_A(real_A), torch.ones_like(D_A(real_A))) +
                    adv_loss(D_A(fake_A.detach()), torch.zeros_like(D_A(fake_A))))
        loss_D_A.backward()
        optimizer_D_A.step()

        # ============================================
        # Train D_B
        # ============================================
        optimizer_D_B.zero_grad()
        loss_D_B = (adv_loss(D_B(real_B), torch.ones_like(D_B(real_B))) +
                    adv_loss(D_B(fake_B.detach()), torch.zeros_like(D_B(fake_B))))
        loss_D_B.backward()
        optimizer_D_B.step()

        loop.set_description(f"Epoch {epoch}/{EPOCHS}")
        loop.set_postfix(G=loss_G.item())

    # ===========================================================
    # SAVE MODEL EACH EPOCH
    # ===========================================================
    torch.save({
        "G_A2B": G_A2B.state_dict(),
        "G_B2A": G_B2A.state_dict(),
        "D_A": D_A.state_dict(),
        "D_B": D_B.state_dict(),
    }, f"{ckpt_dir}/epoch_{epoch}.pth")

    # ===========================================================
    # VALIDATION
    # ===========================================================
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            A = batch['photo'].to(device)
            B = batch['sketch'].to(device)

            recA = G_B2A(G_A2B(A))
            recB = G_A2B(G_B2A(B))

            loss = cycle_loss_fn(recA, A) + cycle_loss_fn(recB, B)
            total += loss.item()

    avg = total / len(val_loader)

    if avg < best_val:
        best_val = avg
        torch.save({
            "G_A2B": G_A2B.state_dict(),
            "G_B2A": G_B2A.state_dict(),
        }, f"{ckpt_dir}/best_model.pth")

# ===============================================================
# SAVE FINAL MODEL FOR UI
# ===============================================================
torch.save({
    "G_A2B": G_A2B.state_dict(),
    "G_B2A": G_B2A.state_dict(),
}, f"{ckpt_dir}/final_model.pth")

print("🎉 FINAL MODEL SAVED!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch 10/10: 100%|██████████| 500/500 [00:46<00:00, 10.84it/s, G=3.54]


🎉 FINAL MODEL SAVED!


In [7]:
import torch

path = "/content/drive/MyDrive/Checkpoints_CycleGAN_Fast/final_model.pth"
ckpt = torch.load(path, map_location="cpu")

print("Keys:", ckpt.keys())


Keys: dict_keys(['G_A2B', 'G_B2A'])


In [8]:
import os

os.makedirs("templates", exist_ok=True)
os.makedirs("static/uploads", exist_ok=True)
os.makedirs("static/results", exist_ok=True)
os.makedirs("model", exist_ok=True)


In [9]:
import os

if os.path.exists("templates/index.html"):
    print("✅ index.html exists!")
else:
    print("❌ index.html NOT found!")


✅ index.html exists!


In [14]:
# ================================
# Setup ngrok with your token
# ================================
from pyngrok import ngrok

# Replace this with your token
NGROK_AUTH_TOKEN = "34nHRQEhWsyECIuOzxFeuaoJFyn_qNnJCPo47LWrPKwpsBKc"

# Set your authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start tunnel
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)


🚀 Public URL: NgrokTunnel: "https://alexandria-contestable-piper.ngrok-free.dev" -> "http://localhost:5000"


In [26]:
# ================================
# Install required packages
# ================================
!pip install flask pyngrok pillow torch torchvision

# ================================
# Imports
# ================================
import os
import io
import base64
from flask import Flask, render_template, request
from pyngrok import ngrok
from PIL import Image
import torch
from torchvision import transforms
import torch.nn as nn

# ================================
# Create templates folder
# ================================
os.makedirs("templates", exist_ok=True)

# ================================
# index.html template
# ================================
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Face <-> Sketch Converter</title>
</head>
<body>
    <h2>Upload a Photo or Sketch</h2>
    <form method="POST" enctype="multipart/form-data" action="/convert">
        <input type="file" name="file" required>
        <button type="submit">Convert</button>
    </form>

    {% if result_img %}
    <h3>Result:</h3>
    <img src="{{ result_img }}" width="256">
    {% endif %}
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(index_html)

# ================================
# Flask app
# ================================
app = Flask(__name__)

# ================================
# Device
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ================================
# Define Model (Resnet + Generator)
# ================================
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, padding=1),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 3, padding=1),
            nn.InstanceNorm2d(dim)
        )
    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_blocks=3):
        super().__init__()
        model = [nn.Conv2d(input_nc, 64, 7, padding=3),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(True)]
        in_f, out_f = 64, 128
        model += [nn.Conv2d(in_f, out_f, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(out_f),
                  nn.ReLU(True)]
        in_f = out_f
        out_f = 256
        model += [nn.Conv2d(in_f, out_f, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(out_f),
                  nn.ReLU(True)]
        in_f = out_f
        for _ in range(n_blocks):
            model += [ResnetBlock(in_f)]
        out_f = in_f // 2
        model += [nn.ConvTranspose2d(in_f, out_f, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(out_f),
                  nn.ReLU(True)]
        in_f = out_f
        out_f = in_f // 2
        model += [nn.ConvTranspose2d(in_f, out_f, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(out_f),
                  nn.ReLU(True)]
        model += [nn.Conv2d(out_f, output_nc, 7, padding=3), nn.Tanh()]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# ================================
# Initialize & Load Models
# ================================
G_A2B = Generator(3,3).to(device)
G_B2A = Generator(3,3).to(device)

ckpt_path = "/content/drive/MyDrive/Checkpoints_CycleGAN_Fast/final_model.pth"
checkpoint = torch.load(ckpt_path, map_location=device)
G_A2B.load_state_dict(checkpoint["G_A2B"])
G_B2A.load_state_dict(checkpoint["G_B2A"])
G_A2B.eval()
G_B2A.eval()

# ================================
# Image Transform
# ================================
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def tensor_to_pil(tensor):
    tensor = tensor.squeeze(0).cpu().detach()
    tensor = (tensor + 1)/2
    tensor = tensor.clamp(0,1)
    return transforms.ToPILImage()(tensor)

# ================================
# Flask Routes
# ================================
@app.route("/", methods=["GET"])
def home():
    return render_template("index.html", result_img=None)

@app.route("/convert", methods=["POST"])
def convert():
    file = request.files["file"]
    img = Image.open(file).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    avg_intensity = img_tensor.mean().item()
    with torch.no_grad():
        if avg_intensity < 0.5:
            output = G_B2A(img_tensor)
        else:
            output = G_A2B(img_tensor)

    out_img = tensor_to_pil(output)
    buf = io.BytesIO()
    out_img.save(buf, format="PNG")
    img_base64 = base64.b64encode(buf.getvalue()).decode("utf-8")

    return render_template("index.html", result_img="data:image/png;base64," + img_base64)

# ================================
# Start Flask via Ngrok
# ================================
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)
app.run()


🚀 Public URL: NgrokTunnel: "https://alexandria-contestable-piper.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:27:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:27:54] "POST /convert HTTP/1.1" 200 -
